This notebook expects a file "raw.tar.gz" to be available. This file is a compressed tarball containing the folder "raw" which is produced by "get_raw_data.py" scripts. 

This notebook produces a "packages.csv" file that contains the metadata (that we need) for each version of each package that was available on PyPi when "get_raw_data.py" was launched.

In [1]:
import os
import json
import tarfile
import pandas

In [2]:
INPUT_FILEPATH = 'data/raw.tar.gz'
OUTPUT_FILEPATH = 'data/packages.csv'

FIELDS = dict(
    info=['name', 'version', 'author', 'home_page', 'licence', 
          'maintainer', 'requires', 'requires_dist', 'install_requires'],  # Last one??
    urls=['size', 'upload_time', 'packagetype', 'downloads']  # urls is a list of dict (in package metadata)
)

In [3]:
def filter_data(data):
    ndata = {}
    
    info = data['info']
    
    try:
        urls = data['urls'][0]
    except IndexError:
        urls = {}
        
    for key in FIELDS['info']:
        ndata['info_{}'.format(key)] = info.get(key, None)
    for key in FIELDS['urls']:
        ndata['urls_{}'.format(key)] = urls.get(key, None)
    return ndata

In [4]:
tar = tarfile.open(INPUT_FILEPATH, mode='r:gz')

data = []
for item in tar.getmembers():
    if item.isfile():
        with tar.extractfile(item) as f:
            binary_data = '\n'.join((x.decode('utf-8') for x in f.readlines()))
            package_data = json.loads(binary_data)
            data.append(filter_data(package_data))

In [6]:
df = pandas.DataFrame.from_dict(data)
df.to_csv(OUTPUT_FILEPATH, index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/packages.csv'